In [1]:
import os
import torch
import torchvision
import torch.nn as nn
from torchvision import transforms
from torchvision.utils import save_image


# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Hyper-parameters
latent_size = 64
hidden_size = 256
image_size = 784
num_epochs = 200
batch_size = 100
sample_dir = 'samples'

# Create a directory if not exists
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Image processing
# transform = transforms.Compose([
#                 transforms.ToTensor(),
#                 transforms.Normalize(mean=(0.5, 0.5, 0.5),   # 3 for RGB channels
#                                      std=(0.5, 0.5, 0.5))])
transform = transforms.Compose([
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.5],   # 1 for greyscale channels
                                     std=[0.5])])

# MNIST dataset
mnist = torchvision.datasets.MNIST(root='../../data/',
                                   train=True,
                                   transform=transform,
                                   download=True)

# Data loader
data_loader = torch.utils.data.DataLoader(dataset=mnist,
                                          batch_size=batch_size, 
                                          shuffle=True)

# Discriminator
D = nn.Sequential(
    nn.Linear(image_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, hidden_size),
    nn.LeakyReLU(0.2),
    nn.Linear(hidden_size, 1),
    nn.Sigmoid())

# Generator 
G = nn.Sequential(
    nn.Linear(latent_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, hidden_size),
    nn.ReLU(),
    nn.Linear(hidden_size, image_size),
    nn.Tanh())

# Device setting
D = D.to(device)
G = G.to(device)

# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)

def denorm(x):
    out = (x + 1) / 2
    return out.clamp(0, 1)

def reset_grad():
    d_optimizer.zero_grad()
    g_optimizer.zero_grad()

# Start training
total_step = len(data_loader)
for epoch in range(num_epochs):
    for i, (images, _) in enumerate(data_loader):
        images = images.reshape(batch_size, -1).to(device)
        
        # Create the labels which are later used as input for the BCE loss
        real_labels = torch.ones(batch_size, 1).to(device)
        fake_labels = torch.zeros(batch_size, 1).to(device)

        # ================================================================== #
        #                      Train the discriminator                       #
        # ================================================================== #

        # Compute BCE_Loss using real images where BCE_Loss(x, y): - y * log(D(x)) - (1-y) * log(1 - D(x))
        # Second term of the loss is always zero since real_labels == 1
        outputs = D(images)
        d_loss_real = criterion(outputs, real_labels)
        real_score = outputs
        
        # Compute BCELoss using fake images
        # First term of the loss is always zero since fake_labels == 0
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        d_loss_fake = criterion(outputs, fake_labels)
        fake_score = outputs
        
        # Backprop and optimize
        d_loss = d_loss_real + d_loss_fake
        reset_grad()
        d_loss.backward()
        d_optimizer.step()
        
        # ================================================================== #
        #                        Train the generator                         #
        # ================================================================== #

        # Compute loss with fake images
        z = torch.randn(batch_size, latent_size).to(device)
        fake_images = G(z)
        outputs = D(fake_images)
        
        # We train G to maximize log(D(G(z)) instead of minimizing log(1-D(G(z)))
        # For the reason, see the last paragraph of section 3. https://arxiv.org/pdf/1406.2661.pdf
        g_loss = criterion(outputs, real_labels)
        
        # Backprop and optimize
        reset_grad()
        g_loss.backward()
        g_optimizer.step()
        
        if (i+1) % 200 == 0:
            print('Epoch [{}/{}], Step [{}/{}], d_loss: {:.4f}, g_loss: {:.4f}, D(x): {:.2f}, D(G(z)): {:.2f}' 
                  .format(epoch, num_epochs, i+1, total_step, d_loss.item(), g_loss.item(), 
                          real_score.mean().item(), fake_score.mean().item()))
    
    # Save real images
    if (epoch+1) == 1:
        images = images.reshape(images.size(0), 1, 28, 28)
        save_image(denorm(images), os.path.join(sample_dir, 'real_images.png'))
    
    # Save sampled images
    fake_images = fake_images.reshape(fake_images.size(0), 1, 28, 28)
    save_image(denorm(fake_images), os.path.join(sample_dir, 'fake_images-{}.png'.format(epoch+1)))

# Save the model checkpoints 
torch.save(G.state_dict(), 'G.ckpt')
torch.save(D.state_dict(), 'D.ckpt')

# x:실제데이터, z:가짜데이터(noise), G(z):generator가 noise를 받고만든 결과, 
# D(x):discriminator가 x를 받은 결과, D(G(z));discriminator가 G(z)(fake data)를 받은 결과 => 1으로 만드는 것이 목표

Epoch [0/200], Step [200/600], d_loss: 0.0449, g_loss: 4.1995, D(x): 0.99, D(G(z)): 0.04
Epoch [0/200], Step [400/600], d_loss: 0.0634, g_loss: 5.3734, D(x): 0.99, D(G(z)): 0.05
Epoch [0/200], Step [600/600], d_loss: 0.0503, g_loss: 5.1612, D(x): 0.98, D(G(z)): 0.03
Epoch [1/200], Step [200/600], d_loss: 0.0465, g_loss: 4.9996, D(x): 0.98, D(G(z)): 0.02
Epoch [1/200], Step [400/600], d_loss: 0.2062, g_loss: 3.4404, D(x): 0.90, D(G(z)): 0.08
Epoch [1/200], Step [600/600], d_loss: 0.1558, g_loss: 4.1699, D(x): 0.94, D(G(z)): 0.08
Epoch [2/200], Step [200/600], d_loss: 0.1856, g_loss: 3.8240, D(x): 0.97, D(G(z)): 0.14
Epoch [2/200], Step [400/600], d_loss: 0.0681, g_loss: 4.8547, D(x): 0.96, D(G(z)): 0.02
Epoch [2/200], Step [600/600], d_loss: 0.2059, g_loss: 4.6754, D(x): 0.93, D(G(z)): 0.06
Epoch [3/200], Step [200/600], d_loss: 0.1726, g_loss: 3.7823, D(x): 0.93, D(G(z)): 0.06
Epoch [3/200], Step [400/600], d_loss: 0.8545, g_loss: 1.9465, D(x): 0.80, D(G(z)): 0.30
Epoch [3/200], Step [

Epoch [30/200], Step [600/600], d_loss: 0.4910, g_loss: 3.1396, D(x): 0.91, D(G(z)): 0.19
Epoch [31/200], Step [200/600], d_loss: 0.2620, g_loss: 4.7148, D(x): 0.88, D(G(z)): 0.05
Epoch [31/200], Step [400/600], d_loss: 0.6040, g_loss: 3.0881, D(x): 0.84, D(G(z)): 0.16
Epoch [31/200], Step [600/600], d_loss: 0.4838, g_loss: 2.9848, D(x): 0.90, D(G(z)): 0.18
Epoch [32/200], Step [200/600], d_loss: 0.4284, g_loss: 3.6634, D(x): 0.87, D(G(z)): 0.13
Epoch [32/200], Step [400/600], d_loss: 0.4763, g_loss: 2.7449, D(x): 0.89, D(G(z)): 0.15
Epoch [32/200], Step [600/600], d_loss: 0.3024, g_loss: 3.5700, D(x): 0.89, D(G(z)): 0.10
Epoch [33/200], Step [200/600], d_loss: 0.5330, g_loss: 3.1850, D(x): 0.94, D(G(z)): 0.26
Epoch [33/200], Step [400/600], d_loss: 0.6107, g_loss: 2.6538, D(x): 0.89, D(G(z)): 0.22
Epoch [33/200], Step [600/600], d_loss: 0.1521, g_loss: 3.8147, D(x): 0.95, D(G(z)): 0.08
Epoch [34/200], Step [200/600], d_loss: 0.3338, g_loss: 4.0758, D(x): 0.93, D(G(z)): 0.13
Epoch [34/

Epoch [61/200], Step [400/600], d_loss: 0.5290, g_loss: 2.3225, D(x): 0.87, D(G(z)): 0.24
Epoch [61/200], Step [600/600], d_loss: 0.7711, g_loss: 1.9827, D(x): 0.82, D(G(z)): 0.26
Epoch [62/200], Step [200/600], d_loss: 0.5303, g_loss: 3.4937, D(x): 0.77, D(G(z)): 0.13
Epoch [62/200], Step [400/600], d_loss: 0.7997, g_loss: 1.7670, D(x): 0.73, D(G(z)): 0.21
Epoch [62/200], Step [600/600], d_loss: 0.6796, g_loss: 2.8748, D(x): 0.76, D(G(z)): 0.17
Epoch [63/200], Step [200/600], d_loss: 0.6203, g_loss: 2.0768, D(x): 0.77, D(G(z)): 0.18
Epoch [63/200], Step [400/600], d_loss: 0.7910, g_loss: 1.9855, D(x): 0.80, D(G(z)): 0.29
Epoch [63/200], Step [600/600], d_loss: 0.5545, g_loss: 1.8236, D(x): 0.87, D(G(z)): 0.25
Epoch [64/200], Step [200/600], d_loss: 0.6494, g_loss: 2.3773, D(x): 0.74, D(G(z)): 0.14
Epoch [64/200], Step [400/600], d_loss: 0.6640, g_loss: 2.4072, D(x): 0.78, D(G(z)): 0.21
Epoch [64/200], Step [600/600], d_loss: 0.5532, g_loss: 2.7396, D(x): 0.87, D(G(z)): 0.25
Epoch [65/

Epoch [92/200], Step [200/600], d_loss: 0.8563, g_loss: 2.1267, D(x): 0.81, D(G(z)): 0.37
Epoch [92/200], Step [400/600], d_loss: 1.0269, g_loss: 1.2762, D(x): 0.71, D(G(z)): 0.29
Epoch [92/200], Step [600/600], d_loss: 0.8234, g_loss: 1.7720, D(x): 0.68, D(G(z)): 0.23
Epoch [93/200], Step [200/600], d_loss: 0.7364, g_loss: 2.6798, D(x): 0.69, D(G(z)): 0.15
Epoch [93/200], Step [400/600], d_loss: 0.8221, g_loss: 2.2837, D(x): 0.73, D(G(z)): 0.25
Epoch [93/200], Step [600/600], d_loss: 0.5809, g_loss: 2.1156, D(x): 0.75, D(G(z)): 0.16
Epoch [94/200], Step [200/600], d_loss: 0.5255, g_loss: 1.6408, D(x): 0.85, D(G(z)): 0.26
Epoch [94/200], Step [400/600], d_loss: 0.8991, g_loss: 1.8658, D(x): 0.66, D(G(z)): 0.19
Epoch [94/200], Step [600/600], d_loss: 0.8124, g_loss: 1.6134, D(x): 0.74, D(G(z)): 0.26
Epoch [95/200], Step [200/600], d_loss: 0.7878, g_loss: 1.4637, D(x): 0.72, D(G(z)): 0.27
Epoch [95/200], Step [400/600], d_loss: 0.7858, g_loss: 1.6169, D(x): 0.73, D(G(z)): 0.26
Epoch [95/

Epoch [122/200], Step [400/600], d_loss: 0.8006, g_loss: 1.4468, D(x): 0.77, D(G(z)): 0.32
Epoch [122/200], Step [600/600], d_loss: 0.9594, g_loss: 1.6550, D(x): 0.71, D(G(z)): 0.34
Epoch [123/200], Step [200/600], d_loss: 0.7396, g_loss: 1.7333, D(x): 0.76, D(G(z)): 0.25
Epoch [123/200], Step [400/600], d_loss: 1.0171, g_loss: 1.8146, D(x): 0.74, D(G(z)): 0.37
Epoch [123/200], Step [600/600], d_loss: 1.2336, g_loss: 1.5160, D(x): 0.49, D(G(z)): 0.16
Epoch [124/200], Step [200/600], d_loss: 0.5946, g_loss: 2.4687, D(x): 0.81, D(G(z)): 0.24
Epoch [124/200], Step [400/600], d_loss: 0.8756, g_loss: 1.4805, D(x): 0.75, D(G(z)): 0.30
Epoch [124/200], Step [600/600], d_loss: 0.8399, g_loss: 2.4682, D(x): 0.71, D(G(z)): 0.24
Epoch [125/200], Step [200/600], d_loss: 0.7696, g_loss: 1.5748, D(x): 0.72, D(G(z)): 0.25
Epoch [125/200], Step [400/600], d_loss: 0.8651, g_loss: 1.7042, D(x): 0.64, D(G(z)): 0.21
Epoch [125/200], Step [600/600], d_loss: 0.8503, g_loss: 2.2404, D(x): 0.67, D(G(z)): 0.22

Epoch [152/200], Step [600/600], d_loss: 0.9611, g_loss: 1.7881, D(x): 0.67, D(G(z)): 0.27
Epoch [153/200], Step [200/600], d_loss: 0.9221, g_loss: 1.1165, D(x): 0.72, D(G(z)): 0.35
Epoch [153/200], Step [400/600], d_loss: 0.8777, g_loss: 2.3416, D(x): 0.64, D(G(z)): 0.19
Epoch [153/200], Step [600/600], d_loss: 1.0943, g_loss: 1.5158, D(x): 0.75, D(G(z)): 0.42
Epoch [154/200], Step [200/600], d_loss: 0.9894, g_loss: 1.7464, D(x): 0.66, D(G(z)): 0.31
Epoch [154/200], Step [400/600], d_loss: 1.0010, g_loss: 1.7934, D(x): 0.69, D(G(z)): 0.31
Epoch [154/200], Step [600/600], d_loss: 0.8397, g_loss: 1.8857, D(x): 0.72, D(G(z)): 0.30
Epoch [155/200], Step [200/600], d_loss: 0.9189, g_loss: 1.8673, D(x): 0.70, D(G(z)): 0.32
Epoch [155/200], Step [400/600], d_loss: 0.8010, g_loss: 1.7158, D(x): 0.73, D(G(z)): 0.28
Epoch [155/200], Step [600/600], d_loss: 0.8724, g_loss: 1.5418, D(x): 0.67, D(G(z)): 0.27
Epoch [156/200], Step [200/600], d_loss: 0.8464, g_loss: 1.6559, D(x): 0.72, D(G(z)): 0.30

Epoch [183/200], Step [200/600], d_loss: 0.9849, g_loss: 1.3113, D(x): 0.72, D(G(z)): 0.38
Epoch [183/200], Step [400/600], d_loss: 1.0101, g_loss: 1.2755, D(x): 0.71, D(G(z)): 0.36
Epoch [183/200], Step [600/600], d_loss: 0.9980, g_loss: 1.4703, D(x): 0.66, D(G(z)): 0.33
Epoch [184/200], Step [200/600], d_loss: 1.0511, g_loss: 1.4942, D(x): 0.69, D(G(z)): 0.35
Epoch [184/200], Step [400/600], d_loss: 0.8859, g_loss: 1.6610, D(x): 0.71, D(G(z)): 0.30
Epoch [184/200], Step [600/600], d_loss: 0.8517, g_loss: 1.7060, D(x): 0.69, D(G(z)): 0.28
Epoch [185/200], Step [200/600], d_loss: 0.8851, g_loss: 1.5476, D(x): 0.68, D(G(z)): 0.27
Epoch [185/200], Step [400/600], d_loss: 1.0173, g_loss: 1.1833, D(x): 0.80, D(G(z)): 0.45
Epoch [185/200], Step [600/600], d_loss: 0.8596, g_loss: 1.6665, D(x): 0.64, D(G(z)): 0.22
Epoch [186/200], Step [200/600], d_loss: 0.9688, g_loss: 1.2775, D(x): 0.72, D(G(z)): 0.36
Epoch [186/200], Step [400/600], d_loss: 0.9932, g_loss: 1.4349, D(x): 0.64, D(G(z)): 0.29